In [2]:
import os

import cv2
import numpy as np
from numpy import random
from scipy.special import binom
import matplotlib as mpl

import matplotlib.pyplot as plt
import tkinter


In [6]:
"""
This file is primary taken from https://stackoverflow.com/questions/50731785/create-random-shape-contour-using-matplotlib
"""
#
import os

import cv2
import numpy as np
from numpy import random
from scipy.special import binom
import matplotlib as mpl

import matplotlib.pyplot as plt
import tkinter

bernstein = lambda n, k, t: binom(n, k) * t ** k * (1. - t) ** (n - k)


def bezier(points, num=200):
    number_of_points = len(points)
    t = np.linspace(0, 1, num=num)
    curve = np.zeros((num, 2))

    # Calculate the outer product of all vectors.
    for i in range(number_of_points):
        curve += np.outer(bernstein(number_of_points - 1, i, t), points[i])
    return curve


class Segment():
    def __init__(self, p1, p2, angle1, angle2, **kw):
        self.p1 = p1;
        self.p2 = p2
        self.angle1 = angle1;
        self.angle2 = angle2
        self.numpoints = kw.get("numpoints", 100)
        r = kw.get("r", 0.3)
        d = np.sqrt(np.sum((self.p2 - self.p1) ** 2))
        self.r = r * d
        self.p = np.zeros((4, 2))
        self.p[0, :] = self.p1[:]
        self.p[3, :] = self.p2[:]
        self.calc_intermediate_points(self.r)

    def calc_intermediate_points(self, r):
        self.p[1, :] = self.p1 + np.array([self.r * np.cos(self.angle1),
                                           self.r * np.sin(self.angle1)])
        self.p[2, :] = self.p2 + np.array([self.r * np.cos(self.angle2 + np.pi),
                                           self.r * np.sin(self.angle2 + np.pi)])
        self.curve = bezier(self.p, self.numpoints)


def get_curve(points, **kw):
    segments = []
    for i in range(len(points) - 1):
        seg = Segment(points[i, :2], points[i + 1, :2], points[i, 2], points[i + 1, 2], **kw)
        segments.append(seg)
    curve = np.concatenate([s.curve for s in segments])
    return segments, curve


def ccw_sort(p):
    d = p - np.mean(p, axis=0)
    s = np.arctan2(d[:, 0], d[:, 1])
    return p[np.argsort(s), :]


def get_bezier_curve(a, rad=0.2, edgy: float = 0):
    """ given an array of points *a*, create a curve through
    those points.
    *rad* is a number between 0 and 1 to steer the distance of
          control points.
    *edgy* is a parameter which controls how "edgy" the curve is,
           edgy=0 is smoothest."""
    p = np.arctan(edgy) / np.pi + .5
    a = ccw_sort(a)
    a = np.append(a, np.atleast_2d(a[0, :]), axis=0)
    d = np.diff(a, axis=0)
    ang = np.arctan2(d[:, 1], d[:, 0])
    f = lambda ang: (ang >= 0) * ang + (ang < 0) * (ang + 2 * np.pi)
    ang = f(ang)
    ang1 = ang
    ang2 = np.roll(ang, 1)
    ang = p * ang1 + (1 - p) * ang2 + (np.abs(ang2 - ang1) > np.pi) * np.pi
    ang = np.append(ang, [ang[0]])
    a = np.append(a, np.atleast_2d(ang).T, axis=1)
    s, c = get_curve(a, r=rad, method="var")
    x, y = c.T
    return x, y, a


def get_random_points(n=4, scale=1.6, mindst=None, rec=0):
    """ create n random points in the unit square, which are *mindst*
    apart, then scale them."""
    mindst = mindst or .7 / n
    a = np.random.rand(n, 2)
    d = np.sqrt(np.sum(np.diff(ccw_sort(a), axis=0), axis=1) ** 2)
    if np.all(d >= mindst) or rec >= 200:
        return a * scale
    else:
        return get_random_points(n=n, scale=scale, mindst=mindst, rec=rec + 1)


def generate(number_of_masks_to_generate: int, path: str, start_index: int = 0, position: tuple = None, size: tuple = (256, 256),
             number_of_points: int = 4, rad: float = 0.5, edgy: float = 0.5, picture_scale: int = None):
    """
    Generates masks
    :param start_index: first image will be named "start_index.png"
    :param path: the path where to save the images.
    :param number_of_masks_to_generate: the number of images to generate.
    :param position: box coordinate where to draw the mask [x1, x2, y1, y2].
    :param size: picture size (width, height).
    :param number_of_points: number of points used to generate the mask, more points means more complex shape.
    :param rad: controls the angles.
    :param edgy: controls how smooth is the shape. 0 is the smoothest.
    :param picture_scale: proportion of the mask compared to the image size.
    :return:
    """
    mpl.use("Agg")
    mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=["#FFFFFF"])
    root = tkinter.Tk()
    my_dpi = root.winfo_fpixels('1i')

    (w, h) = size
    random_scale = False
    random_position = False
    random_point_number = False

    # Set scale, point_number and position to random if no input exists
    if picture_scale is None:
        random_scale = True
    if position is None:
        random_position = True
    if number_of_points is None:
        random_point_number = True

    for index in range(start_index, number_of_masks_to_generate):
        if random_scale:
            picture_scale = random.randint(low=20, high=90)

        if random_position:
            # Calculate the minimum height and width needed
            min_width_needed = int(w / 100 * picture_scale)
            min_height_needed = int(h / 100 * picture_scale)
            #
            starting_point_x = random.randint(10, w - min_width_needed - 10)
            starting_point_y = random.randint(10, h - min_height_needed - 10)

            position = [starting_point_x, starting_point_x + min_width_needed, starting_point_y,
                        starting_point_y + min_height_needed]

        if random_point_number:
            number_of_points = random.randint(8, 26)

        a = get_random_points(n=number_of_points, scale=1)
        x, y, _ = get_bezier_curve(a, rad=rad, edgy=edgy)
        x = np.hstack((x,x))
        y = np.hstack((y,y))
        
        # Draw first patch and save it
        fig = plt.figure(figsize=(w / my_dpi, h / my_dpi), dpi=my_dpi)
        fig.patch.set_facecolor('#000000')
        plt.ioff()
        plt.axis('off')
        plt.fill_between(x, y)
        plt.plot(x, y, '#FFFFFF')
        fig.savefig("tmp.png", dpi=my_dpi)

        # Load the patch
        image = cv2.imread("tmp.png")

        # Scale it and plot it.
        fig2 = plt.figure(figsize=(w / my_dpi, h / my_dpi), dpi=my_dpi)
        fig2.patch.set_facecolor('#000000')
        ax = plt.axes([0, 0, 1, 1], frameon=False)
        ax.set_xlim(0, w)
        ax.set_ylim(0, h)
        plt.axis('off')
        plt.imshow(image, extent=position)

        fig2.savefig(os.path.join(path, str(index) + ".png"), dpi=my_dpi)
        os.remove("tmp.png")

generate(2048 ,"./bezier1024/" )


In [7]:
import skimage.io as skio
from PIL import Image

path = './bezier1024/'
labels = []
num_img  = 0
for file in os.listdir(path):
    print(path+file)
    img = skio.imread(path+file)
    img = np.hstack((img,img))
    print(img.shape)
    imgn = Image.fromarray(img)
    
    imgn.save(f"./test/{num_img}.png")
    num_img = num_img+1

./bezier1024/0.png
(256, 512, 4)
./bezier1024/1.png
(256, 512, 4)
./bezier1024/10.png
(256, 512, 4)
./bezier1024/100.png
(256, 512, 4)
./bezier1024/1000.png
(256, 512, 4)
./bezier1024/1001.png
(256, 512, 4)
./bezier1024/1002.png
(256, 512, 4)
./bezier1024/1003.png
(256, 512, 4)
./bezier1024/1004.png
(256, 512, 4)
./bezier1024/1005.png
(256, 512, 4)
./bezier1024/1006.png
(256, 512, 4)
./bezier1024/1007.png
(256, 512, 4)
./bezier1024/1008.png
(256, 512, 4)
./bezier1024/1009.png
(256, 512, 4)
./bezier1024/101.png
(256, 512, 4)
./bezier1024/1010.png
(256, 512, 4)
./bezier1024/1011.png
(256, 512, 4)
./bezier1024/1012.png
(256, 512, 4)
./bezier1024/1013.png
(256, 512, 4)
./bezier1024/1014.png
(256, 512, 4)
./bezier1024/1015.png
(256, 512, 4)
./bezier1024/1016.png
(256, 512, 4)
./bezier1024/1017.png
(256, 512, 4)
./bezier1024/1018.png
(256, 512, 4)
./bezier1024/1019.png
(256, 512, 4)
./bezier1024/102.png
(256, 512, 4)
./bezier1024/1020.png
(256, 512, 4)
./bezier1024/1021.png
(256, 512, 4)
./b

./bezier1024/1203.png
(256, 512, 4)
./bezier1024/1204.png
(256, 512, 4)
./bezier1024/1205.png
(256, 512, 4)
./bezier1024/1206.png
(256, 512, 4)
./bezier1024/1207.png
(256, 512, 4)
./bezier1024/1208.png
(256, 512, 4)
./bezier1024/1209.png
(256, 512, 4)
./bezier1024/121.png
(256, 512, 4)
./bezier1024/1210.png
(256, 512, 4)
./bezier1024/1211.png
(256, 512, 4)
./bezier1024/1212.png
(256, 512, 4)
./bezier1024/1213.png
(256, 512, 4)
./bezier1024/1214.png
(256, 512, 4)
./bezier1024/1215.png
(256, 512, 4)
./bezier1024/1216.png
(256, 512, 4)
./bezier1024/1217.png
(256, 512, 4)
./bezier1024/1218.png
(256, 512, 4)
./bezier1024/1219.png
(256, 512, 4)
./bezier1024/122.png
(256, 512, 4)
./bezier1024/1220.png
(256, 512, 4)
./bezier1024/1221.png
(256, 512, 4)
./bezier1024/1222.png
(256, 512, 4)
./bezier1024/1223.png
(256, 512, 4)
./bezier1024/1224.png
(256, 512, 4)
./bezier1024/1225.png
(256, 512, 4)
./bezier1024/1226.png
(256, 512, 4)
./bezier1024/1227.png
(256, 512, 4)
./bezier1024/1228.png
(256, 51

(256, 512, 4)
./bezier1024/141.png
(256, 512, 4)
./bezier1024/1410.png
(256, 512, 4)
./bezier1024/1411.png
(256, 512, 4)
./bezier1024/1412.png
(256, 512, 4)
./bezier1024/1413.png
(256, 512, 4)
./bezier1024/1414.png
(256, 512, 4)
./bezier1024/1415.png
(256, 512, 4)
./bezier1024/1416.png
(256, 512, 4)
./bezier1024/1417.png
(256, 512, 4)
./bezier1024/1418.png
(256, 512, 4)
./bezier1024/1419.png
(256, 512, 4)
./bezier1024/142.png
(256, 512, 4)
./bezier1024/1420.png
(256, 512, 4)
./bezier1024/1421.png
(256, 512, 4)
./bezier1024/1422.png
(256, 512, 4)
./bezier1024/1423.png
(256, 512, 4)
./bezier1024/1424.png
(256, 512, 4)
./bezier1024/1425.png
(256, 512, 4)
./bezier1024/1426.png
(256, 512, 4)
./bezier1024/1427.png
(256, 512, 4)
./bezier1024/1428.png
(256, 512, 4)
./bezier1024/1429.png
(256, 512, 4)
./bezier1024/143.png
(256, 512, 4)
./bezier1024/1430.png
(256, 512, 4)
./bezier1024/1431.png
(256, 512, 4)
./bezier1024/1432.png
(256, 512, 4)
./bezier1024/1433.png
(256, 512, 4)
./bezier1024/1434

(256, 512, 4)
./bezier1024/1618.png
(256, 512, 4)
./bezier1024/1619.png
(256, 512, 4)
./bezier1024/162.png
(256, 512, 4)
./bezier1024/1620.png
(256, 512, 4)
./bezier1024/1621.png
(256, 512, 4)
./bezier1024/1622.png
(256, 512, 4)
./bezier1024/1623.png
(256, 512, 4)
./bezier1024/1624.png
(256, 512, 4)
./bezier1024/1625.png
(256, 512, 4)
./bezier1024/1626.png
(256, 512, 4)
./bezier1024/1627.png
(256, 512, 4)
./bezier1024/1628.png
(256, 512, 4)
./bezier1024/1629.png
(256, 512, 4)
./bezier1024/163.png
(256, 512, 4)
./bezier1024/1630.png
(256, 512, 4)
./bezier1024/1631.png
(256, 512, 4)
./bezier1024/1632.png
(256, 512, 4)
./bezier1024/1633.png
(256, 512, 4)
./bezier1024/1634.png
(256, 512, 4)
./bezier1024/1635.png
(256, 512, 4)
./bezier1024/1636.png
(256, 512, 4)
./bezier1024/1637.png
(256, 512, 4)
./bezier1024/1638.png
(256, 512, 4)
./bezier1024/1639.png
(256, 512, 4)
./bezier1024/164.png
(256, 512, 4)
./bezier1024/1640.png
(256, 512, 4)
./bezier1024/1641.png
(256, 512, 4)
./bezier1024/1642

./bezier1024/1824.png
(256, 512, 4)
./bezier1024/1825.png
(256, 512, 4)
./bezier1024/1826.png
(256, 512, 4)
./bezier1024/1827.png
(256, 512, 4)
./bezier1024/1828.png
(256, 512, 4)
./bezier1024/1829.png
(256, 512, 4)
./bezier1024/183.png
(256, 512, 4)
./bezier1024/1830.png
(256, 512, 4)
./bezier1024/1831.png
(256, 512, 4)
./bezier1024/1832.png
(256, 512, 4)
./bezier1024/1833.png
(256, 512, 4)
./bezier1024/1834.png
(256, 512, 4)
./bezier1024/1835.png
(256, 512, 4)
./bezier1024/1836.png
(256, 512, 4)
./bezier1024/1837.png
(256, 512, 4)
./bezier1024/1838.png
(256, 512, 4)
./bezier1024/1839.png
(256, 512, 4)
./bezier1024/184.png
(256, 512, 4)
./bezier1024/1840.png
(256, 512, 4)
./bezier1024/1841.png
(256, 512, 4)
./bezier1024/1842.png
(256, 512, 4)
./bezier1024/1843.png
(256, 512, 4)
./bezier1024/1844.png
(256, 512, 4)
./bezier1024/1845.png
(256, 512, 4)
./bezier1024/1846.png
(256, 512, 4)
./bezier1024/1847.png
(256, 512, 4)
./bezier1024/1848.png
(256, 512, 4)
./bezier1024/1849.png
(256, 51

./bezier1024/2030.png
(256, 512, 4)
./bezier1024/2031.png
(256, 512, 4)
./bezier1024/2032.png
(256, 512, 4)
./bezier1024/2033.png
(256, 512, 4)
./bezier1024/2034.png
(256, 512, 4)
./bezier1024/2035.png
(256, 512, 4)
./bezier1024/2036.png
(256, 512, 4)
./bezier1024/2037.png
(256, 512, 4)
./bezier1024/2038.png
(256, 512, 4)
./bezier1024/2039.png
(256, 512, 4)
./bezier1024/204.png
(256, 512, 4)
./bezier1024/2040.png
(256, 512, 4)
./bezier1024/2041.png
(256, 512, 4)
./bezier1024/2042.png
(256, 512, 4)
./bezier1024/2043.png
(256, 512, 4)
./bezier1024/2044.png
(256, 512, 4)
./bezier1024/2045.png
(256, 512, 4)
./bezier1024/2046.png
(256, 512, 4)
./bezier1024/2047.png
(256, 512, 4)
./bezier1024/205.png
(256, 512, 4)
./bezier1024/206.png
(256, 512, 4)
./bezier1024/207.png
(256, 512, 4)
./bezier1024/208.png
(256, 512, 4)
./bezier1024/209.png
(256, 512, 4)
./bezier1024/21.png
(256, 512, 4)
./bezier1024/210.png
(256, 512, 4)
./bezier1024/211.png
(256, 512, 4)
./bezier1024/212.png
(256, 512, 4)
./b

(256, 512, 4)
./bezier1024/40.png
(256, 512, 4)
./bezier1024/400.png
(256, 512, 4)
./bezier1024/401.png
(256, 512, 4)
./bezier1024/402.png
(256, 512, 4)
./bezier1024/403.png
(256, 512, 4)
./bezier1024/404.png
(256, 512, 4)
./bezier1024/405.png
(256, 512, 4)
./bezier1024/406.png
(256, 512, 4)
./bezier1024/407.png
(256, 512, 4)
./bezier1024/408.png
(256, 512, 4)
./bezier1024/409.png
(256, 512, 4)
./bezier1024/41.png
(256, 512, 4)
./bezier1024/410.png
(256, 512, 4)
./bezier1024/411.png
(256, 512, 4)
./bezier1024/412.png
(256, 512, 4)
./bezier1024/413.png
(256, 512, 4)
./bezier1024/414.png
(256, 512, 4)
./bezier1024/415.png
(256, 512, 4)
./bezier1024/416.png
(256, 512, 4)
./bezier1024/417.png
(256, 512, 4)
./bezier1024/418.png
(256, 512, 4)
./bezier1024/419.png
(256, 512, 4)
./bezier1024/42.png
(256, 512, 4)
./bezier1024/420.png
(256, 512, 4)
./bezier1024/421.png
(256, 512, 4)
./bezier1024/422.png
(256, 512, 4)
./bezier1024/423.png
(256, 512, 4)
./bezier1024/424.png
(256, 512, 4)
./bezier1

(256, 512, 4)
./bezier1024/611.png
(256, 512, 4)
./bezier1024/612.png
(256, 512, 4)
./bezier1024/613.png
(256, 512, 4)
./bezier1024/614.png
(256, 512, 4)
./bezier1024/615.png
(256, 512, 4)
./bezier1024/616.png
(256, 512, 4)
./bezier1024/617.png
(256, 512, 4)
./bezier1024/618.png
(256, 512, 4)
./bezier1024/619.png
(256, 512, 4)
./bezier1024/62.png
(256, 512, 4)
./bezier1024/620.png
(256, 512, 4)
./bezier1024/621.png
(256, 512, 4)
./bezier1024/622.png
(256, 512, 4)
./bezier1024/623.png
(256, 512, 4)
./bezier1024/624.png
(256, 512, 4)
./bezier1024/625.png
(256, 512, 4)
./bezier1024/626.png
(256, 512, 4)
./bezier1024/627.png
(256, 512, 4)
./bezier1024/628.png
(256, 512, 4)
./bezier1024/629.png
(256, 512, 4)
./bezier1024/63.png
(256, 512, 4)
./bezier1024/630.png
(256, 512, 4)
./bezier1024/631.png
(256, 512, 4)
./bezier1024/632.png
(256, 512, 4)
./bezier1024/633.png
(256, 512, 4)
./bezier1024/634.png
(256, 512, 4)
./bezier1024/635.png
(256, 512, 4)
./bezier1024/636.png
(256, 512, 4)
./bezier

./bezier1024/825.png
(256, 512, 4)
./bezier1024/826.png
(256, 512, 4)
./bezier1024/827.png
(256, 512, 4)
./bezier1024/828.png
(256, 512, 4)
./bezier1024/829.png
(256, 512, 4)
./bezier1024/83.png
(256, 512, 4)
./bezier1024/830.png
(256, 512, 4)
./bezier1024/831.png
(256, 512, 4)
./bezier1024/832.png
(256, 512, 4)
./bezier1024/833.png
(256, 512, 4)
./bezier1024/834.png
(256, 512, 4)
./bezier1024/835.png
(256, 512, 4)
./bezier1024/836.png
(256, 512, 4)
./bezier1024/837.png
(256, 512, 4)
./bezier1024/838.png
(256, 512, 4)
./bezier1024/839.png
(256, 512, 4)
./bezier1024/84.png
(256, 512, 4)
./bezier1024/840.png
(256, 512, 4)
./bezier1024/841.png
(256, 512, 4)
./bezier1024/842.png
(256, 512, 4)
./bezier1024/843.png
(256, 512, 4)
./bezier1024/844.png
(256, 512, 4)
./bezier1024/845.png
(256, 512, 4)
./bezier1024/846.png
(256, 512, 4)
./bezier1024/847.png
(256, 512, 4)
./bezier1024/848.png
(256, 512, 4)
./bezier1024/849.png
(256, 512, 4)
./bezier1024/85.png
(256, 512, 4)
./bezier1024/850.png
(2